In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas
from transformers import (AutoModelForCausalLM, AutoTokenizer, BertForSequenceClassification, BertTokenizer, BertModel,
 RobertaForSequenceClassification, RobertaTokenizer, RobertaModel, TrainingArguments, Trainer)


from datasets import load_dataset, load_from_disk, Dataset, DatasetDict, concatenate_datasets
import evaluate
import wandb
import os

os.environ["WANDB_PROJECT"] = "<my-amazing-project>"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints

# Init Generator and Detector

In [3]:
#GEN_PATH = "microsoft/phi-2"
#GEN_PATH = "openai-community/gpt2"
GEN_PATH = "Qwen/Qwen1.5-0.5B-Chat"
#BERT_PATH = "bert-base-uncased"
BERT_PATH = "openai-community/roberta-base-openai-detector"
device = "cuda" if torch.cuda.is_available() else "cpu"


class GPTGenerator(nn.Module):
  def __init__(self, gpt_model, tokenizer):
    super().__init__()

    # gpt should already be trained
    self.gpt = gpt_model
    self.tokenizer = tokenizer

  def forward(self, text, max_length=512, temperature=1, top_k=50, top_p=0.9, repetition_penalty=1, skip_special_tokens=True):
    # tokenize text using the tokenizer
    input_ids = self.tokenizer.encode(text, return_tensors="pt").to(device)
    # generate text using the gpt model
    output_ids = self.gpt.generate(input_ids, max_length=max_length, temperature=temperature, top_k=top_k, top_p=top_p, repetition_penalty=repetition_penalty)

    # optional, remove input_ids from output_ids
    #output_ids = [output_id[len(input_ids):] for input_id, output_id in zip(input_ids, output_ids)]

    # decode the generated text
    decoded_output = self.tokenizer.batch_decode(output_ids, skip_special_tokens=skip_special_tokens)[0]

    #decoded_output = decoded_output.replace(text, "")
    return decoded_output

class BertClassifier(nn.Module):
  def __init__(self, bert_model, tokenizer, num_classes):
    super().__init__()

    self.tokenizer = tokenizer

    # bert should already be trained
    self.bert = bert_model

    # set num_classes
    self.num_classes = num_classes

  def forward(self, text):

    # tokenize text using the tokenizer
    output = self.tokenizer(text, return_tensors="pt")
    input_ids = output["input_ids"].to(device)
    logits = self.bert(input_ids)["logits"]

    # apply sigmoid to get probabilities of each class
    output = torch.sigmoid(logits)
    return output
        

In [3]:
# this causes an issue with Trainer
#torch.set_default_device("cuda")

gen_model = AutoModelForCausalLM.from_pretrained(GEN_PATH, torch_dtype="auto").to(device)
gen_tokenizer = AutoTokenizer.from_pretrained(GEN_PATH, trust_remote_code=True)
generator = GPTGenerator(gen_model, gen_tokenizer)

text_input = '''def print_prime(n):
   """
   Print all primes between 1 and n
   """'''

output = generator(text_input)
print(output)


In [4]:
detector_model = RobertaForSequenceClassification.from_pretrained(BERT_PATH).to(device)
bert_tokenizer = RobertaTokenizer.from_pretrained(BERT_PATH)
detector = BertClassifier(detector_model, bert_tokenizer, 2)

text = "def print_prime(n):\n   \"\"\"\n   Print all primes between 1 and n\n   \"\"\""

logits = detector(text)
fake = logits.argmax().item()

Some weights of the model checkpoint at openai-community/roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
fake

1

# Load dataset of instructions and output with gen

In [5]:
dataset_path = "databricks/databricks-dolly-15k"
dataset = load_dataset(dataset_path)

dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})

In [7]:
dataset["train"][0]

{'instruction': 'When did Virgin Australia start operating?',
 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.",
 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
 'category': 'closed_qa'}

In [11]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [12]:
type(dataset["train"])

datasets.arrow_dataset.Dataset

## Data exploration

In [13]:
# compute mean length of the responses
lengths_response = [len(x) for x in dataset["train"]["response"]]
print("Average length of responses:", np.mean(lengths_response))

Average length of responses: 358.10419026047566


In [14]:
lengts_instruction = [len(x) for x in dataset["train"]["instruction"]]
print("Average length of instructions:", np.mean(lengts_instruction))

Average length of instructions: 71.83938445140231


In [15]:
# discard instructions that are more than max_nb_tokens_input tokens
max_nb_tokens_input = 100

# tokenize the instructions
dataset = dataset.map(lambda x: {"tokenized_instruction": gen_tokenizer(x["instruction"])})
dataset = dataset.map(lambda x: {"tokenized_context": gen_tokenizer(x["context"])})
dataset_before_len = len(dataset["train"])
dataset = dataset.filter(lambda x: len(x["tokenized_instruction"]["input_ids"]) + len(x["tokenized_context"]["input_ids"]) <= max_nb_tokens_input)
dataset_after_len = len(dataset["train"])
print(f"Percent of data discarded: {100*(1 - dataset_after_len/dataset_before_len):.2f}%")

Percent of data discarded: 26.29%


In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category', 'tokenized_instruction', 'tokenized_context'],
        num_rows: 11065
    })
})

In [17]:
# test output with first instruction
text = dataset["train"][0]
text


{'instruction': 'Which is a species of fish? Tope or Rope',
 'context': '',
 'response': 'Tope',
 'category': 'classification',
 'tokenized_instruction': {'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'input_ids': [23085, 374, 264, 9419, 315, 7640, 30, 2014, 375, 476, 97896]},
 'tokenized_context': {'attention_mask': [], 'input_ids': []}}

In [18]:
text_instruction = f"Context: {text["context"]} \n Question: {text["instruction"]}"
output = generator(text_instruction)
print("Question: ", text_instruction)
#print()
print("Generated answer: ", output)
#print()
print("Real human answer: ", text["response"])


Question:  Context:  
 Question: Which is a species of fish? Tope or Rope
Generated answer:  Context:  
 Question: Which is a species of fish? Tope or Rope
Real human answer:  Tope


In [19]:
output = generator("What is the capital of France?")
output

'What is the capital of France?'

In [20]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]
text = gen_tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [21]:
output = generator(text)
print(output)

system
You are a helpful assistant.
user
What is the capital of France?
assistant
Paris


In [22]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"{text_instruction}"},
]
text = gen_tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

output = generator(text)
print(output)

system
You are a helpful assistant.
user
Context:  
 Question: Which is a species of fish? Tope or Rope
assistant
Tope is a species of fish commonly referred to as the "teal fish" or "teal fish." It is a popular aquarium fish species due to its smooth scales and sleek appearance, and it is also known for its strong immune system. rope is a type of fishing line used to catch fish in both freshwater and saltwater environments.


## generate fake dataset

see Open AI GPT-2 generated dataset: https://github.com/openai/gpt-2-output-dataset?tab=readme-ov-file
and https://github.com/openai/gpt-2-output-dataset/tree/master/detector for their roberta detector

In [6]:
def generate_fake_responses(generator, dataset):
    """
    Traverse dataset and generate responses for each instruction
    """

    fake_responses = []
    for data in dataset:
        # Create query in the format that the generator expects
        text_instruction = f"Context: {data['context']} \n {data['instruction']}"
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{text_instruction}"},
        ]
        text = gen_tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Generate response
        output = generator(text, skip_special_tokens=False)
        
        fake_responses.append(output)
    return fake_responses

def create_random_subset(dataset, n=10):
    """
    Create a random subset of the dataset
    """
    if n > len(dataset):
        n = len(dataset)
    indices = np.random.choice(len(dataset), n, replace=False)
    subset = dataset.select(indices)
    return subset

def filter_instruction(sample):
    """
    Note: only works if special tokens are not removed
    """

    text_instruction = f"Context: {sample['context']} \n {sample['instruction']}"
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"{text_instruction}"},
    ]
    text_template = gen_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    generated_response = sample["generated_response"]

    response_without_instruction = generated_response.replace(text_template, "")
    return {"generated_response": response_without_instruction}

In [24]:
# take a random subset of the dataset
subset_size = 5
train_subset = create_random_subset(dataset["train"], n=subset_size)
#test_subset = create_random_subset(dataset["test"], n=10)
#eval_subset = create_random_subset(dataset["validation"], n=10)

# generate fake responses for the subsets
fake_responses_train = generate_fake_responses(generator, train_subset)
#fake_responses_test = generate_fake_responses(generator, test_subset)
#fake_responses_eval = generate_fake_responses(generator, eval_subset)

#fake_dataset = Dataset.from_dict({"train": fake_responses_train, "test": fake_responses_test, "validation": fake_responses_eval})

fake_responses_train = Dataset.from_dict({"generated_response": fake_responses_train, "instruction": train_subset["instruction"],
    "context": train_subset["context"], "true_response": train_subset["response"], "category": train_subset["category"]})

fake_dataset = DatasetDict()
fake_dataset["train"] = fake_responses_train

# save fake dataset
fake_dataset.save_to_disk("fake_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/5 [00:00<?, ? examples/s]

In [12]:
gen_tokenizer = AutoTokenizer.from_pretrained(GEN_PATH, trust_remote_code=True)

# load fake dataset
fake_dataset = load_from_disk("fake_dataset")
fake_dataset

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


DatasetDict({
    train: Dataset({
        features: ['generated_response', 'instruction', 'context', 'true_response', 'category'],
        num_rows: 5
    })
})

In [13]:
print(fake_dataset["train"][0]["instruction"])

Why do humans like dogs?


In [14]:
print(fake_dataset["train"][0]["true_response"])

Humans like dogs because they make us feel good. When a human pets a dog, oxytocin levels rise in both species, which make both feel emotionally closer. These feelings of closeness make us want to pet them more – and the dog wanting to be petted more – increasing feelings of closeness and connectedness while engaging in this biological and emotional feedback loop.


In [15]:
print(fake_dataset["train"][0]["generated_response"])

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Context:  
 Why do humans like dogs?<|im_end|>
<|im_start|>assistant
 Humans enjoy dogs because they are loyal, affectionate, and typically intelligent pets. Dogs have been around for thousands of years and have a long history of being dogs in various forms, from small, shih-tzu to large, golden retrievers. Dogs can also be taught different behaviors and personalities through training.

One of the reasons why humans like dogs is that dogs are social animals. Dogs require social interaction and are able to form strong bonds with their owners. They also make excellent companions, helping humans stay focused and relaxed while they spend time with their owners.

Another reason why humans like dogs is that they are fun to play with. Dogs come in a variety of sizes, breeds, and colors, making them perfect for all types of dog owners. Dogs are also known for their affection and创造性 play.

Finally, dogs are highly intell

In [16]:
fake_dataset = fake_dataset.map(filter_instruction)

In [17]:
print(fake_dataset["train"][0]["generated_response"])

 Humans enjoy dogs because they are loyal, affectionate, and typically intelligent pets. Dogs have been around for thousands of years and have a long history of being dogs in various forms, from small, shih-tzu to large, golden retrievers. Dogs can also be taught different behaviors and personalities through training.

One of the reasons why humans like dogs is that dogs are social animals. Dogs require social interaction and are able to form strong bonds with their owners. They also make excellent companions, helping humans stay focused and relaxed while they spend time with their owners.

Another reason why humans like dogs is that they are fun to play with. Dogs come in a variety of sizes, breeds, and colors, making them perfect for all types of dog owners. Dogs are also known for their affection and创造性 play.

Finally, dogs are highly intelligent animals that can be trained to perform various tasks, such as hunting, searching, and serving as therapy dogs. They can also teach humans 

In [18]:
true_dataset = dataset.select_columns(["response"])
true_dataset = true_dataset.rename_column("response", "text")

# select random samples from true_dataset to match fake_dataset size
true_dataset = true_dataset.shuffle(seed=42)
true_dataset = true_dataset.select(range(len(fake_dataset["train"])))

# create label = 0 for true responses and label = 1 for fake responses
true_dataset = true_dataset.map(lambda x: {"label": 0})
fake_dataset = fake_dataset.map(lambda x: {"label": 1})

# remove true_response from fake_dataset
fake_dataset = fake_dataset.remove_columns(["true_response"])

# rename generated_response to response
fake_dataset = fake_dataset.rename_column("generated_response", "text")
fake_dataset = fake_dataset.select_columns(["text", "label"])

# merge fake and true datasets
merged = concatenate_datasets([true_dataset["train"], fake_dataset["train"]])
merged_dataset = DatasetDict()
merged_dataset["train"] = merged

# shuffle the dataset
merged_dataset = merged_dataset.shuffle(seed=42)

# save merged dataset
merged_dataset.save_to_disk("merged_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/15016 [00:00<?, ? examples/s]

In [19]:
# load merged dataset
human_fake_train_dataset = load_from_disk("merged_dataset")

In [20]:
true_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 15011
    })
})

In [21]:
fake_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5
    })
})

In [22]:
merged_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 15016
    })
})

# Training Detector

In [23]:
# tokenize the merged dataset with the detector tokenizer
def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length")

human_fake_train_dataset = human_fake_train_dataset.map(lambda x: tokenize_text(x, bert_tokenizer), batched=True)


In [24]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=1e-3,
    logging_steps=5,
    logging_dir="./logs",
    report_to="wandb"
)

trainer = Trainer(
    model=detector_model,
    args=training_args,
    train_dataset=human_fake_train_dataset["train"],
    #eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: hdasilva. Use `wandb login --relogin` to force relogin


Step,Training Loss
5,3.803000
10,2.367800
15,0.834400
20,0.779000
25,2.246600
30,2.046300
35,1.158100
40,0.919600
45,0.213500
50,0.060200


KeyboardInterrupt: 

# DPO training BERT

In [ ]:
training_args = TrainingArguments(
    output_dir="./output",
    report_to="wandb",
    logging_dir="./logs",
    logging_steps=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=1e-3,
    num_train_epochs=3,
)

dpo_trainer = DPOTrainer(
    model,
    model_ref=None,
    args=training_args,
    beta=0.1,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

dpo_trainer.train()
wandb.finish()